In [1]:
pip install autogluon


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for autogluon from https://files.pythonhosted.org/packages/2e/42/f26592ecf3dc4e4edcbc5f7f7e3deba25b0681fad2ad04a82d12af17e3a4/autogluon-0.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.core[all]==0.8.2 from https://files.pythonhosted.org/packages/e0/56/545adb1d388e78591cd7e36de0c8b889c1944de362bdaeec0f31d01890df/autogluon.core-0.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.features==0.8.2 from https://files.pythonhosted.org/packages/bb/ea/7892719f78a30aee1bf42c4a0540fbae98bfbdf56b85fab79ffc437eb687/autogluon.features-0.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.tabular[all]==0.8.2 from https://files.pythonhosted.org/packages/f3/dc/0bd8cadb9a5e2f3e5b12caaa6745357d912ffc7b8b75fb4e426a38331028/autogluon.tabular-0.8.2-py3-none-any.whl.metadata
  Obtaining dependency information 

In [6]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

# Cargar datos
data = TabularDataset('/home/marchesini_id/buckets/b1/datasets/competencia_03.csv.gz')

# Filtrar datos para excluir los meses mencionados
train_months = [202001, 202002, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107]
test_months = [202109]





In [7]:
import os
#creamos clase binaria
data.loc[:, 'clase01'] = (data['clase_ternaria'] != 'CONTINUA').astype(int)

#creamos campos buenos
campos_buenos = set(data.columns) - set(["clase_ternaria", "clase01"])

#separamos lo que vamos a usar como train
data['train'] = 0
data.loc[data['foto_mes'].isin(train_months), 'train'] = 1

#creamos ruta donde va el experimento
os.makedirs("/home/marchesini_id/buckets/b1/exp/AutoGluon_test_2/", exist_ok=True)

# Establezco el directorio de trabajo DEL EXPERIMENTO
os.chdir("/home/marchesini_id/buckets/b1/exp/AutoGluon_test_2/")
save_path = '/home/marchesini_id/buckets/b1/exp/AutoGluon_test_2/'


train_data = data[data['train'] == 1]
X_train = train_data[campos_buenos]
y_train = train_data['clase01']


/tmp/ipykernel_2267/195046191.py:21: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_train = train_data[campos_buenos]


In [ ]:
import pandas as pd

#Data para autogluon
df = pd.concat([X_train, y_train], axis = 1)
df.head()

In [ ]:
train_data_autogluon = TabularDataset(df)
label_column = 'clase01'
predictor = TabularPredictor(label=label_column, path = save_path).fit(train_data_autogluon, excluded_model_types = ['KNN', 'XT'], presets = 'good_quality', hyperparameters = 'light')

Presets specified: ['good_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "/home/marchesini_id/buckets/b1/exp/AutoGluon_test_2/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #14~22.04.1-Ubuntu SMP Mon Sep 11 07:44:43 UTC 2023
Disk Space Avail:   1125899.91 GB / 1125899.91 GB (100.0%)
Train Data Rows:    2073987
Train Data Columns: 640
Label Column: clase01
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the probl

	0.9904	 = Validation score   (accuracy)
	807.6s	 = Training   runtime
	21.07s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Memory not enough to fit LGBModel folds in parallel. Will do sequential fitting instead. 	Consider decreasing folds trained in parallel by passing num_folds_parallel to ag_args_ensemble when calling predictor.fit
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	0.9905	 = Validation score   (accuracy)
	691.73s	 = Training   runtime
	20.35s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L1 ...
	0.9903	 = Validation score   (accuracy)
	888.39s	 = Training   runtime
	547.22s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L1 ...
	0.9903	 = Validation score   (accuracy)
	924.23s	 = Training   runtime
	556.24s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Memory not enough to fit CatBoostModel folds in parallel. Will do sequential fitting instead. 	Consider decreasing folds trained in 

In [13]:
resultados = predictor.fit_summary() 
#resultados.to_csv('/home/marchesini_id/buckets/b1/exp/AutoGluon_test_2/restuladosResumen.csv')

*** Summary of fit() ***
Estimated performance of each model:
                           model  score_val  pred_time_val       fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0            WeightedEnsemble_L2   0.990541     272.281715  137026.334636                3.784367         454.351182            2      False         10
1          NeuralNetTorch_BAG_L1   0.990524     203.694392    6048.247561              203.694392        6048.247561            1      False          8
2                 XGBoost_BAG_L1   0.990518      64.802957  130523.735893               64.802957      130523.735893            1      False          7
3                LightGBM_BAG_L1   0.990459      20.345138     691.732774               20.345138         691.732774            1      False          2
4              LightGBMXT_BAG_L1   0.990440      21.073280     807.603339               21.073280         807.603339            1      False          1
5           LightGBMLarge_

/home/marchesini_id/.local/lib/python3.10/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [14]:
#Testeo
dapply = data[data['foto_mes'] == 202109]
X_dapply = dapply[campos_buenos]
prediccion = predictor.predict_proba(X_dapply)

tabla_entrega = dapply[['numero_de_cliente','foto_mes']].copy()
#y_pred = predictor.predict(test_data_nolab)
#print("Predictions:  \n", y_pred)
#perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


/tmp/ipykernel_2267/3853982863.py:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_dapply = dapply[campos_buenos]


In [15]:
tabla_entrega

,numero_de_cliente,foto_mes
32,29215143,202109
47,29219583,202109
80,29221308,202109
113,29287764,202109
146,29307944,202109
...,...,...
4893017,186627721,202109
4893021,187108474,202109
4893024,187451677,202109
4893026,187647553,202109


In [16]:
tabla_entrega['prob_0'] = prediccion[0]
tabla_entrega['prob_1'] = prediccion[1]
tabla_entrega

,numero_de_cliente,foto_mes,prob_0,prob_1
32,29215143,202109,0.999987,0.000013
47,29219583,202109,0.949688,0.050312
80,29221308,202109,0.937427,0.062573
113,29287764,202109,0.999874,0.000126
146,29307944,202109,0.999915,0.000085
...,...,...,...,...
4893017,186627721,202109,0.969549,0.030451
4893021,187108474,202109,0.997754,0.002246
4893024,187451677,202109,0.996381,0.003619
4893026,187647553,202109,0.997474,0.002526


In [17]:
tabla_entrega.to_csv('/home/marchesini_id/buckets/b1/exp/AutoGluon_test_2/predicciones_autogluon.csv', index = False)

In [34]:
kaggle_entrega = tabla_entrega[['numero_de_cliente','foto_mes', 'prob_1']].copy()
kaggle_entrega.to_csv('/home/marchesini_id/buckets/b1/exp/AutoGluon_test_2/predicciones_kaggle_autogluon.csv', index = False)